In [137]:
import copy
import itertools
from collections import defaultdict
from operator import itemgetter

In [138]:
#DataSet Format
#An event is a list of String
#A sequence is a list of events
#A dataset is a list of sequences
#Thus, a dataset is a list of lists of lists of strings

dataset = [
    [["a"],["a","b","c"],["a","c"],["c"]],
    [["a"],["c"],["a","c"]],
    [["a","b"],["d"],["c"],["b"],["c"]],
    [["a"],["c"],["b"],["c"]]
]


In [139]:
#Foundations Functions

#1.This fun is a recursive method that checks if subsequence is a subSequence
#of mainSequence

def isSubsequence(mainSequence:list,subSequence:list,pos=0):
    subSequence_copy = copy.deepcopy(subSequence)
    if(subSequence_copy==[]):
        return True
    element = set(subSequence_copy.pop(0))
    for i in range(pos,len(mainSequence)):
        if(set(mainSequence[i]).issuperset(element)):
            return isSubsequence(mainSequence,subSequence_copy,i+1)
    return False


In [140]:
#Test Function isSubsequence

testMainSeq = [["a"],["b","c"],["d"],["e"],["a","e"]]
print(isSubsequence(testMainSeq,[["a"],["d"],["e"]]))
print(isSubsequence(testMainSeq,[["d"],["b"]]))


True
False


In [141]:
#2.This fun is to calculate the length of the seq
def sequenceLength(sequence):
    return sum(len(i) for i in sequence)


In [142]:
#Test Function sequenceLength
print(sequenceLength([["a","b"],["c"],["d"],["e","f","g"]]))



7


In [143]:
#3.This fun is to calculate the support of a sequence is a dataset
def countSupport(dataset,sequence):
    return sum(1 for seq in dataset if isSubsequence(seq,sequence))



In [144]:
#Test Function countSupport
print(countSupport(dataset=dataset,sequence=[["a"]]))


4


In [145]:
##############################################
#Algorithm 1:AprioriAll based Pattern Mining #
##############################################

In [146]:
#1.From the core of Apriori mining, we need to get length k candidate from two
# candidates of length k-1

#Which kind of two k-1 length candidates can be merged into a k length candidate?

#If two candidates look like:
#cand1 = a(body)
#cand2 = (body)b
#Then these two candidates can be merged into a k+1 length candidate:
#cand3 = a(body)b

def generateCandidate(candidate1:list,candidate2:list):
    cand1 = copy.deepcopy(candidate1)
    cand2 = copy.deepcopy(candidate2)
    #The principle is to judge the body part of two candidates is equal or not

    #drop the head of candidate1
    #When the first element of cand1 has only 1 item
    if(len(cand1[0])==1):
        cand1.pop(0)
    else:
    #When there are multi items in the first element of cand1
        cand1[0] = cand1[0][1:]

    #drop the tail of candidate2
    #When the last element of cand2 has only 1 item
    if(len(cand2[-1])==1):
        cand2.pop(-1)
    else:
    #When there are multi items in the last element of cand2
        cand2[-1] = cand2[-1][:-1]

    #If the body part is not equal
    if cand1 != cand2:
        return []
    else:
        candidate3 = copy.deepcopy(candidate1)
        if(len(candidate2[-1])==1):
            candidate3.append(candidate2[-1])
        else:
            candidate3[-1].extend(candidate2[-1][-1])
        return candidate3


In [147]:
#Test fun generateCandidate
candA = [["a"],["b","c"],["d"]]
candB = [["b","c"],["d","e"]]
print(generateCandidate(candA,candB))

candC = [["a"],["b","c"],["d"]]
candD = [["b","c"],["d"],["e"]]
print(generateCandidate(candC,candD))

candE = [["a"],["b","c"],["e","f"]]
candF = [["a"],["b","c"],["f"]]
print(generateCandidate(candE,candF))

[['a'], ['b', 'c'], ['d', 'e']]
[['a'], ['b', 'c'], ['d'], ['e']]
[]


In [148]:
#2.The we need to decide how to get the k+1 length candidate set from k length
#candidate set
def getCandidatesSet(lastLevelCandidates:list):
    k = sequenceLength(lastLevelCandidates[0])+1

    #If it is from the begining, which means k = 1 and k + 1 = 2
    if(k==2):
        print("Start from begining")
        flattenCandidates = [item for list in lastLevelCandidates for sublist in list for item in sublist]
        result = [[[a,b]] for a in flattenCandidates for b in flattenCandidates if b>a]
        result.extend([[a],[b]] for a in flattenCandidates for b in flattenCandidates)
        return result
    else:
        result = []
        for i in range(0,len(lastLevelCandidates)):
            for j in range(0,len(lastLevelCandidates)):
                newCand = generateCandidate(lastLevelCandidates[i],lastLevelCandidates[j])
                if(newCand!=[]):
                    result.append(newCand)
        result.sort()
        return result




In [149]:
#Test fun getCandidatesSet
lastLevelCand1 = [
    [['a']],
    [['b']],
    [['c']],
    [['d']]
]
print(getCandidatesSet(lastLevelCand1))


lastLevelCand2 = [
    [['a','b']],
    [['b','c']],
    [['a'],['b']],
    [['c'],['d']],
    [['a'],['a']]
]

print(getCandidatesSet(lastLevelCand2))


Start from begining
[[['a', 'b']], [['a', 'c']], [['a', 'd']], [['b', 'c']], [['b', 'd']], [['c', 'd']], [['a'], ['a']], [['a'], ['b']], [['a'], ['c']], [['a'], ['d']], [['b'], ['a']], [['b'], ['b']], [['b'], ['c']], [['b'], ['d']], [['c'], ['a']], [['c'], ['b']], [['c'], ['c']], [['c'], ['d']], [['d'], ['a']], [['d'], ['b']], [['d'], ['c']], [['d'], ['d']]]
[[['a'], ['a'], ['a']], [['a'], ['a'], ['b']], [['a'], ['a', 'b']], [['a'], ['b', 'c']], [['a', 'b', 'c']], [['b', 'c'], ['d']]]


In [150]:
#3.As we want to keep the longest sequence pattern rather than keep all subsequence.
#So We need to find out the longest one and delete subsequence of it from the final
#result

#The first thing we will do is to find k-1 length subsequence from a k length sequence

def findSubsequence(sequence:list):
    subsequence = []
    #delete exactly one item from sequence
    for i,itemset in enumerate(sequence):
        if(len(itemset)==1):
            sequence_copy = copy.deepcopy(sequence)
            sequence_copy.pop(i)
            subsequence.append(sequence_copy)
        else:
            for j in range(len(itemset)):
                sequence_copy = copy.deepcopy(sequence)
                sequence_copy[i].pop(j)
                subsequence.append(sequence_copy)
    return subsequence

#Then we will prunes all k-1 length sequence which generate from k length sequence
def pruneSubsequence(lastpatterns,newpatterns):
    return [candidate for candidate in newpatterns if all(sub in lastpatterns for sub in findSubsequence(candidate))]


In [151]:
#Test prune fun
newseq = getCandidatesSet(lastLevelCand2)
print(newseq)
result = pruneSubsequence(lastLevelCand2,newseq)
print(result)



[[['a'], ['a'], ['a']], [['a'], ['a'], ['b']], [['a'], ['a', 'b']], [['a'], ['b', 'c']], [['a', 'b', 'c']], [['b', 'c'], ['d']]]
[[['a'], ['a'], ['a']], [['a'], ['a'], ['b']], [['a'], ['a', 'b']]]


In [152]:
#4.Then we want to filter by minSupport
def filterByMinSup(minSpport,pruneSeq,dataset):
    candCount = [(seq,countSupport(dataset,seq)) for seq in pruneSeq]
    result = [(seq,count) for (seq,count) in candCount if (count >= minSpport)]
    return result


In [153]:
#Test fun filter by minSUp
lastseq=[
    [['a','b']],
    [['b','c']],
    [['a'],['b']],
    [['a'],['c']],
    [['b'],['c']],
    [['c'],['b']],
    [['c'],['c']]
]
nowseq = getCandidatesSet(lastseq)
print(nowseq)
pruneseq = pruneSubsequence(lastseq,newseq)
print(pruneseq)
result = filterByMinSup(0,pruneseq,dataset)
print(result)


[[['a'], ['b'], ['c']], [['a'], ['b', 'c']], [['a'], ['c'], ['b']], [['a'], ['c'], ['c']], [['a', 'b'], ['c']], [['a', 'b', 'c']], [['b'], ['c'], ['b']], [['b'], ['c'], ['c']], [['b', 'c'], ['b']], [['b', 'c'], ['c']], [['c'], ['b'], ['c']], [['c'], ['b', 'c']], [['c'], ['c'], ['b']], [['c'], ['c'], ['c']]]
[[['a'], ['b', 'c']]]
[([['a'], ['b', 'c']], 1)]
